In [1]:
import random
import itertools
import numpy
import math
import sys

In [2]:
def calcular_matriz_de_costes(dataset):
    n_ciudades = len(dataset)
    distancias = numpy.zeros((n_ciudades,n_ciudades))
    for i in range(n_ciudades):
        for j in range(n_ciudades):
            if(i != j):
                distancias[i][j] = calcular_distancia_euclidea(dataset[i],dataset[j])
    return distancias

In [3]:
def calcular_distancia_euclidea(ciudad_1,ciudad_2):
    
    x1,y1 = ciudad_1[1 :]
    x2,y2 = ciudad_2[1 :]
    
    xd = x1 - x2
    yd = y1 - y2
    
    return round(math.sqrt( xd*xd + yd*yd ))

In [4]:
def calcular_coste(camino,distancias):
    coste = 0
    for indice in range(len(camino)-1):
        coste += distancias[camino[indice]][camino[indice+1]]
    coste += distancias[camino[-1]][camino[0]]
    return coste

In [5]:
def generar_array_ciudades_cercanas_a_vecinos(combinaciones_indices_vecinos,len_dataset):
    ciudades_cercanas = []

    for combinacion in combinaciones_indices_vecinos:
        if (combinacion[0] == 0):
            a1 = len_dataset - 1
        else:
            a1 = combinacion[0] - 1

        if (combinacion[0] == len_dataset - 1):
            s1 = 0
        else:
            s1 = combinacion[0] + 1

        if (combinacion[1] == 0):
            a2 = len_dataset - 1
        else:
            a2 = combinacion[1] - 1

        if (combinacion[1] == len_dataset - 1):
            s2 = 0
        else:
            s2 = combinacion[1] + 1

        ciudades_cercanas.append([a1, s1, a2, s2])

    return ciudades_cercanas

In [6]:
def calcular_coste_intercambio(mejor_coste,mejor_solucion,vecinos,intercambio,len_dataset,matriz_de_costes):
    # Por cada vecino, estudiaremos su coste
    coste_candidato = mejor_coste

    # Primero, por comodidad, generaremos los indices del camino
    a1 = vecinos[0]
    s1 = vecinos[1]
    a2 = vecinos[2]
    s2 = vecinos[3]
    i1 = intercambio[0]
    i2 = intercambio[1]

    if (i2 - 1 == i1):

        # A continuación eliminaremos el coste generado por las conexiones de nodos a intercambiar

        coste_candidato -= matriz_de_costes[mejor_solucion[a1]][mejor_solucion[i1]]
        coste_candidato -= matriz_de_costes[mejor_solucion[i1]][mejor_solucion[i2]]
        coste_candidato -= matriz_de_costes[mejor_solucion[i2]][mejor_solucion[s2]]

        # Despues sumamos el coste del nuevo camino

        coste_candidato += matriz_de_costes[mejor_solucion[a1]][mejor_solucion[i2]]
        coste_candidato += matriz_de_costes[mejor_solucion[i2]][mejor_solucion[i1]]
        coste_candidato += matriz_de_costes[mejor_solucion[i1]][mejor_solucion[s2]]

    elif(i1 == 0 and i2 == len_dataset-1):
        # A continuación eliminaremos el coste generado por las conexiones de nodos a intercambiar

        coste_candidato -= matriz_de_costes[mejor_solucion[a2]][mejor_solucion[i2]]
        coste_candidato -= matriz_de_costes[mejor_solucion[i2]][mejor_solucion[i1]]
        coste_candidato -= matriz_de_costes[mejor_solucion[i1]][mejor_solucion[s1]]

        # Despues sumamos el coste del nuevo camino

        coste_candidato += matriz_de_costes[mejor_solucion[a2]][mejor_solucion[i1]]
        coste_candidato += matriz_de_costes[mejor_solucion[i1]][mejor_solucion[i2]]
        coste_candidato += matriz_de_costes[mejor_solucion[i2]][mejor_solucion[s1]]
    else:

        # A continuación eliminaremos el coste generado por las conexiones de nodos a intercambiar

        coste_candidato -= matriz_de_costes[mejor_solucion[a1]][mejor_solucion[i1]]
        coste_candidato -= matriz_de_costes[mejor_solucion[i1]][mejor_solucion[s1]]

        coste_candidato -= matriz_de_costes[mejor_solucion[a2]][mejor_solucion[i2]]
        coste_candidato -= matriz_de_costes[mejor_solucion[i2]][mejor_solucion[s2]]

        # Despues sumamos el coste del nuevo camino

        coste_candidato += matriz_de_costes[mejor_solucion[a1]][mejor_solucion[i2]]
        coste_candidato += matriz_de_costes[mejor_solucion[i2]][mejor_solucion[s1]]

        coste_candidato += matriz_de_costes[mejor_solucion[a2]][mejor_solucion[i1]]
        coste_candidato += matriz_de_costes[mejor_solucion[i1]][mejor_solucion[s2]]
    
    return coste_candidato

In [7]:
def busqueda_local_el_primer_mejor_vecino(semilla, dataset):
    
    random.seed(semilla)                                                # Se establece la semilla inicial
    len_dataset = len(dataset)                                          # Tamaño dataset
    matriz_de_costes = calcular_matriz_de_costes(dataset)               # Generación de matriz de costes
    mejor_solucion = random.sample(range(len_dataset),len_dataset)      # Generación de solución inicial
    mejor_coste = calcular_coste(mejor_solucion, matriz_de_costes)      # Generación del coste inicial

    # Número máximo de evaluaciones, segunda condición de parada del bucle.
    num_evaluacion = len_dataset*1600

    # Indices de todos los posibles intercambios de dos ciudades del dataset [ (0,1), (0,2), ... , (10,23), ...  ]
    intercambios_de_dos_ciudades = list(itertools.combinations(list(range(0, len_dataset)), 2))
    len_intercambios_de_dos_ciudades = len(intercambios_de_dos_ciudades)

    # Indices de los vecinos de los indices del array intercambios_de_dos_ciudades : 
    # intercambios_de_dos_ciudades[x] = (6,10)
    # vecinos_de_intercambios_de_dos_ciudades = [5,7,9,11]
    vecinos_de_intercambios_de_dos_ciudades = generar_array_ciudades_cercanas_a_vecinos(intercambios_de_dos_ciudades,len_dataset)

    
    seguir = True       # Variable de salida del bucle
    evaluacion = 0      # Nº de evaluaciones actual del bucle

    while (seguir):

        intercambio_actual = ()
        coste_actual = mejor_coste

        for i in range(len_intercambios_de_dos_ciudades):

            coste_candidato = calcular_coste_intercambio(mejor_coste,mejor_solucion,vecinos_de_intercambios_de_dos_ciudades[i],intercambios_de_dos_ciudades[i],len_dataset,matriz_de_costes)

            evaluacion += 1
            
            if (coste_candidato < coste_actual):
                coste_actual = coste_candidato
                intercambio_actual = intercambios_de_dos_ciudades[i]
                break # El unico cambio respecto a busqueda local el mejor

            if evaluacion >= num_evaluacion:
                break
                
        if intercambio_actual == ():
            seguir = False
        else:
            buffer = mejor_solucion[intercambio_actual[0]]
            mejor_solucion[intercambio_actual[0]] = mejor_solucion[intercambio_actual[1]]
            mejor_solucion[intercambio_actual[1]] = buffer
            mejor_coste = coste_actual

        if evaluacion >= num_evaluacion:
            seguir = False

    return [dataset[ciudad] for ciudad in mejor_solucion], mejor_coste, evaluacion